# Recommendation using clustering | Collaberative method

In [1]:
import pandas as pd
import numpy as np

**Creating a pivot table**

In [2]:
books=pd.read_csv('Books.csv')
books

,bookid,title,author,published_year,publisher
0,195153448,Classical Mythology,Mark P O Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo DEste,1991,HarperPerennial
3,374157065,Flu The Story of the Great Influenza Pandemic ...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E J W Barber,1999,W W Norton amp Company
...,...,...,...,...,...
266393,440400988,Theres a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub Mm
266394,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
266395,6008667,Lily Dale The True Story of the Town that Tal...,Christine Wicker,2004,HarperSanFrancisco
266396,192126040,Republic Worlds Classics,Plato,1996,Oxford University Press


In [3]:
books[books.duplicated(keep=False)]

,bookid,title,author,published_year,publisher


In [4]:
books[books['bookid']==88404632]

,bookid,title,author,published_year,publisher
37838,88404632,Dianetics The Modern Science of Mental Health,L Ron Hubbard,1995,Bridge Publications CA


In [5]:
books.iloc[64]

bookid                                                    345260317
title             The Dragons of Eden Speculations on the Evolut...
author                                                   Carl Sagan
published_year                                                 1978
publisher                                          Ballantine Books
Name: 64, dtype: object

In [6]:
books.drop_duplicates(inplace=True)
books

,bookid,title,author,published_year,publisher
0,195153448,Classical Mythology,Mark P O Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo DEste,1991,HarperPerennial
3,374157065,Flu The Story of the Great Influenza Pandemic ...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E J W Barber,1999,W W Norton amp Company
...,...,...,...,...,...
266393,440400988,Theres a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub Mm
266394,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
266395,6008667,Lily Dale The True Story of the Town that Tal...,Christine Wicker,2004,HarperSanFrancisco
266396,192126040,Republic Worlds Classics,Plato,1996,Oxford University Press


In [7]:
rating=pd.read_csv('BookRatings.csv')
rating

,userid,bookid,rating
0,276725,34545104,5
1,276726,155061224,3
2,276727,446520802,5
3,276729,52165615,2
4,276729,521795028,3
...,...,...,...
999887,240144,373262345,5
999888,240144,373262787,5
999889,240144,37326285,5
999890,240144,37326335,5


In [8]:
rating['userid'].value_counts()

userid
11676     13588
198711     7548
153662     6109
98391      5891
35859      5850
          ...  
100339        1
100343        1
100344        1
100347        1
118510        1
Name: count, Length: 91407, dtype: int64

When one user has rated so many books, it skews the system towards that single user's ratings. Hence to dissolve that bias, we 

In [9]:
#Filter out users who have rated more than 200 books 
morethan200=rating['userid'].value_counts() > 200
morethan200[morethan200].shape

(774,)

In [10]:
morethan200ind=morethan200[morethan200].index
morethan200ind

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       116122,  33145,  88793,   9856,  28634, 155916,  44296,  59727, 188951,
        73681],
      dtype='int64', name='userid', length=774)

In [11]:
rating=rating[rating['userid'].isin(morethan200ind)]
rating

,userid,bookid,rating
1455,277427,2542730,5
1456,277427,26217457,5
1457,277427,3008685,5
1458,277427,30615321,5
1459,277427,60002050,5
...,...,...,...
998340,239594,9061683432,5
998341,239594,9067661023,4
998342,239594,9067661171,4
998343,239594,9072103211,5


In [12]:
rating.shape

(461575, 3)

In [13]:
rating_books=rating.merge(books, on='bookid', how='inner')

In [14]:
rating_books

,userid,bookid,rating,title,author,published_year,publisher
0,277427,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc
1,3363,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc
2,11676,2542730,3,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc
3,12538,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc
4,13552,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc
...,...,...,...,...,...,...,...
423008,239594,9053336230,3,Doch verder geen paniek Doorgrondelijke wijshe...,Herman Brusselmans,1997,Prometheus
423009,239594,905333842,5,Uitgeverij Guggenheimer,Herman Brusselmans,1999,Prometheus
423010,239594,9053339248,4,Vergeef mij de liefde,Herman Brusselmans,2000,Prometheus
423011,239594,9061683041,5,De hoed van tante Jeannot Taferelen uit de kin...,Eric de Kuyper,1989,SUN


In [15]:
rating_books[rating_books.duplicated(keep=False)]

,userid,bookid,rating,title,author,published_year,publisher


In [16]:
num_rat=rating_books.groupby('title')['rating'].count().reset_index()
num_rat

,title,rating
0,A Novel,1
1,A Nutshell handbook,1
2,Always Have Popsicles,1
3,And for Starters,1
4,Apple Magic The Collectors series,1
...,...,...
146435,voyage fatal,1
146436,wet sand raven tracks,1
146437,whataboutrickcom a poetic tribute to Richard A...,1
146438,why Im like this True Stories,4


In [17]:
num_rat.rename(columns={'rating':'Number_rating'}, inplace=True)
num_rat

,title,Number_rating
0,A Novel,1
1,A Nutshell handbook,1
2,Always Have Popsicles,1
3,And for Starters,1
4,Apple Magic The Collectors series,1
...,...,...
146435,voyage fatal,1
146436,wet sand raven tracks,1
146437,whataboutrickcom a poetic tribute to Richard A...,1
146438,why Im like this True Stories,4


In [18]:
final_rat=rating_books.merge(num_rat, on='title')
final_rat

,userid,bookid,rating,title,author,published_year,publisher,Number_rating
0,277427,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
1,3363,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
2,11676,2542730,3,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
3,12538,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
4,13552,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
...,...,...,...,...,...,...,...,...
423008,239594,9053336230,3,Doch verder geen paniek Doorgrondelijke wijshe...,Herman Brusselmans,1997,Prometheus,1
423009,239594,905333842,5,Uitgeverij Guggenheimer,Herman Brusselmans,1999,Prometheus,1
423010,239594,9053339248,4,Vergeef mij de liefde,Herman Brusselmans,2000,Prometheus,1
423011,239594,9061683041,5,De hoed van tante Jeannot Taferelen uit de kin...,Eric de Kuyper,1989,SUN,1


In [19]:
#Only keep more than 50 rating
final_rat=final_rat[final_rat['Number_rating']>=50]
final_rat

,userid,bookid,rating,title,author,published_year,publisher,Number_rating
0,277427,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
1,3363,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
2,11676,2542730,3,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
3,12538,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
4,13552,2542730,5,Politically Correct Bedtime Stories Modern Tal...,James Finn Garner,1994,John Wiley amp Sons Inc,72
...,...,...,...,...,...,...,...,...
201697,198699,743439651,5,Still Waters,Jennifer Lauck,2001,Atria,70
201698,225087,743439651,5,Still Waters,Jennifer Lauck,2001,Atria,70
201699,127233,61090956,5,Still Waters,Kerry Tucker,1992,Harper Mass Market Paperbacks Mm,70
201700,178667,74350996,5,Still Waters,Jennifer Lauck,2001,Simon amp Schuster Audio,70


In [20]:
final_rat.drop_duplicates(['userid','title'], inplace=True)

/var/folders/06/cby1pvh535d6qwj22pwqn7080000gn/T/ipykernel_68276/1127019100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rat.drop_duplicates(['userid','title'], inplace=True)


In [21]:
final_rat.shape

(45209, 8)

Pivot Table

In [22]:
pivot_book=final_rat.pivot_table(columns='userid', index='title', values='rating')
pivot_book

userid,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,238699,238781,238864,238961,239584,239594,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year of Wonders,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
pivot_book.shape

(582, 763)

In [25]:
pivot_book.fillna(0, inplace=True)
pivot_book

userid,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,238699,238781,238864,238961,239584,239594,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Starting the training
Don't calculate 0 since 0 here means empty 

In [50]:
from scipy.sparse import csr_matrix

In [51]:
book_sparse=csr_matrix(pivot_book)
book_sparse

<582x763 sparse matrix of type '<class 'numpy.float64'>'
	with 45209 stored elements in Compressed Sparse Row format>

Clustering algorithm

In [52]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [53]:
model.fit(book_sparse) 
# This algorithm returns the distance and the suggestion

NearestNeighbors(algorithm='brute')

In [ ]:
def recommendBook(book_title):  
    # Check if the title exists in the DataFrame index
    if book_title in pivot_book.index:
        book_index = pivot_book.index.get_loc(book_title)
        dist, suggest = model.kneighbors(pivot_book.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)
        for i in range(len(suggest)):
            newBooks = pivot_book.index[suggest[i]]
            for j in newBooks:
                print(j)
    else:
        print("Book name not found")
        

In [54]:
ind=getIndex('Harry Potter and the Chamber of Secrets Book 2')
if ind!=-1:
    dist, suggest = model.kneighbors(pivot_book.iloc[ind, :].values.reshape(1, -1), n_neighbors=6)

In [55]:
dist

array([[ 0.        , 45.607017  , 46.06517123, 54.69917732, 55.02726597,
        56.54201977]])

In [56]:
suggest

array([[178, 181, 179, 180, 182, 183]])

In [57]:
for i in range(len(suggest)):
    print(pivot_book.index[suggest[i]])

Index(['Harry Potter and the Chamber of Secrets Book 2',
       'Harry Potter and the Prisoner of Azkaban Book 3',
       'Harry Potter and the Goblet of Fire Book 4',
       'Harry Potter and the Order of the Phoenix Book 5',
       'Harry Potter and the Sorcerers Stone Book 1',
       'Harry Potter and the Sorcerers Stone Harry Potter Paperback'],
      dtype='object', name='title')


In [58]:
namesBook=pivot_book.index
namesBook

Index(['1984', '1st to Die A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called It One Childs Courage to Survive', 'A Civil Action',
       'A Fine Balance', 'A Heartbreaking Work of Staggering Genius',
       ...
       'Wild Animus', 'Winter Moon', 'Winter Solstice', 'Wish You Well',
       'Without Remorse', 'Wuthering Heights', 'Year of Wonders',
       'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance An Inquiry into Values',
       'Zoya'],
      dtype='object', name='title', length=582)

In [61]:
recommendBook('Harry Potter and the Chamber of Secrets Book 2')

Harry Potter and the Chamber of Secrets Book 2
Harry Potter and the Prisoner of Azkaban Book 3
Harry Potter and the Goblet of Fire Book 4
Harry Potter and the Order of the Phoenix Book 5
Harry Potter and the Sorcerers Stone Book 1
Harry Potter and the Sorcerers Stone Harry Potter Paperback


In [62]:
recommendBook('Lost Ark')

Book name not found


In [64]:
recommendBook('Zoya')

Zoya
Heartbeat
Fine Things
Kaleidoscope
Daddy
No Greater Love
